In [ ]:
# notes - visualisations for report?

## RF Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import json
import shiny_data
import pandas as pd
from PIL import Image, ImageEnhance
from skimage.feature import hog
from sklearn.decomposition import PCA
from IPython.display import display
from sklearn.metrics import classification_report

# suppress warnings for results
import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

Xmat_train, Xmat_val, Xmat_tests_0, Xmat_tests_1, Xmat_tests_2, y_train_enc, y_val_enc, y_tests_enc_0, y_tests_enc_1, y_tests_enc_2 = shiny_data.load_split_images()

def store_results(results, y_true, y_pred, probs=None, blur=0, noise=0, hog=False, pca=False, **extra_metrics):        
    entry = {
        "blur_size": blur,
        "noise_level": noise,
        "HOG": hog,
        "PCA": pca,
        "accuracy": accuracy_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred, average='macro'),
        "precision": precision_score(y_true, y_pred, average='macro'),
        "recall": recall_score(y_true, y_pred, average='macro'),
    }

    f1_classes = f1_score(y_true, y_pred, average=None, zero_division=0)
    precision_classes = precision_score(y_true, y_pred, average=None, zero_division=0)
    recall_classes = recall_score(y_true, y_pred, average=None, zero_division=0)

    label_map = {
        0: "immune",
        1: "other",
        2: "stromal",
        3: "tumour"
    }
    
    for i in range(4):
        label = label_map[i]
        entry[f"f1_{label}"] = f1_classes[i]
        entry[f"precision_{label}"] = precision_classes[i]
        entry[f"recall_{label}"] = recall_classes[i]
    
    if probs is not None:
        confidences = np.max(probs, axis=1)
        entry["confidence_overall"] = np.mean(confidences)
        for cls in range(4):
            label = label_map[cls]
            cls_conf = confidences[y_pred == cls]
            entry[f"confidence_{label}_avg"] = np.mean(cls_conf) if len(cls_conf) > 0 else np.nan
            entry[f"confidence_{label}_std"] = np.std(cls_conf) if len(cls_conf) > 0 else np.nan
    
    total_preds = len(y_pred)
    for cls in range(4):
        label = label_map[cls]
        entry[f"count_pred_{label}"] = np.sum(y_pred == cls)

    # store actual and predicted labels and the confusion matrix as strings
    entry["confusion_matrix"] = str(confusion_matrix(y_true, y_pred, labels=[0, 1, 2, 3]).tolist())

    entry.update(extra_metrics)
    results.append(entry)

def extract_hog_features(images):
    hog_features = []
    for img in images:
        if img.shape[-1] == 3:
            img = Image.fromarray((img * 255).astype(np.uint8)).convert("L")
            img = np.array(img)
        features = hog(img, pixels_per_cell=(8, 8), cells_per_block=(2, 2), feature_vector=True)
        hog_features.append(features)
    return np.array(hog_features)
    

# preprocessing
X_train_flat = Xmat_train.reshape(Xmat_train.shape[0], -1)
X_val_flat = Xmat_val.reshape(Xmat_val.shape[0], -1)
pca = PCA(n_components=100)
X_train_pca = pca.fit_transform(X_train_flat)
X_train_hog = extract_hog_features(Xmat_train)

# fitting models
model_base = RandomForestClassifier(n_estimators=100, random_state=3888)
model_base.fit(X_train_flat, y_train_enc)

model_pca = RandomForestClassifier(n_estimators=100, random_state=3888)
model_pca.fit(X_train_pca, y_train_enc)

model_hog = RandomForestClassifier(n_estimators=100, random_state=3888)
model_hog.fit(X_train_hog, y_train_enc)

# helper functions, model map
def evaluate_and_store(model, X, y_true, probs=True, **kwargs):
    y_pred = model.predict(X)
    y_probs = model.predict_proba(X) if probs else None
    store_results(results, y_true, y_pred, probs=y_probs, **kwargs)

models = {
    "base": (model_base, lambda X: X.reshape(X.shape[0], -1), {}),
    "pca": (model_pca, lambda X: pca.transform(X.reshape(X.shape[0], -1)), {"pca": True}),
    "hog": (model_hog, extract_hog_features, {"hog": True}),
}

results = []

def run_all_variants(X_base, y_base):
    blur_levels = [0, 1, 3, 5, 7, 9, 19]
    noise_levels = [0, 1, 3, 5, 10, 20, 30]

    # Unmodified input (baseline)
    print("▶️  Evaluating baseline (no blur, no noise)...")
    for name, (model, transform, flags) in models.items():
        print(f"   🔹 Model: {name}")
        X_trans = transform(X_base)
        evaluate_and_store(model, X_trans, y_base, **flags)

    # Blur only
    for blur in blur_levels:
        print(f"▶️  Evaluating blur={blur}, noise=0...")
        X_blur = shiny_data.apply_blur(X_base, blur)
        for name, (model, transform, flags) in models.items():
            print(f"   🔹 Model: {name}")
            X_trans = transform(X_blur)
            evaluate_and_store(model, X_trans, y_base, blur=blur, **flags)

    # Noise only
    for noise in noise_levels:
        print(f"▶️  Evaluating blur=0, noise={noise}...")
        np.random.seed(3888 + noise)
        X_noisy = shiny_data.apply_noise(X_base, std=noise)
        for name, (model, transform, flags) in models.items():
            print(f"   🔹 Model: {name}")
            X_trans = transform(X_noisy)
            evaluate_and_store(model, X_trans, y_base, noise=noise, **flags)

    # Blur + noise 
    for blur in blur_levels:
        for noise in noise_levels:
            print(f"▶️  Evaluating blur={blur}, noise={noise}...")
            np.random.seed(10000 + blur * 100 + noise)
            X_blur = shiny_data.apply_blur(X_base, blur)
            X_combo = shiny_data.apply_noise(X_blur, std=noise)
            for name, (model, transform, flags) in models.items():
                print(f"   🔹 Model: {name}")
                X_trans = transform(X_combo)
                evaluate_and_store(model, X_trans, y_base, blur=blur, noise=noise, **flags)

test_sets = [
    ("test0", Xmat_tests_0, y_tests_enc_0),
    ("test1", Xmat_tests_1, y_tests_enc_1),
    ("test2", Xmat_tests_2, y_tests_enc_2),
]

for name, X, y in test_sets:
    print(f"\n🧪 Running evaluations on test set: {name}")
    run_all_variants(X, y)

pd.DataFrame(results).to_csv("rf_evaluation_results.csv", index=False)

[('Immune', 0), ('Other', 1), ('Stromal', 2), ('Tumour', 3)]

🧪 Running evaluations on test set: test0
▶️  Evaluating baseline (no blur, no noise)...
   🔹 Model: base
   🔹 Model: pca
   🔹 Model: hog
▶️  Evaluating blur=0, noise=0...
   🔹 Model: base
   🔹 Model: pca
   🔹 Model: hog
▶️  Evaluating blur=1, noise=0...
   🔹 Model: base
   🔹 Model: pca
   🔹 Model: hog
▶️  Evaluating blur=3, noise=0...
   🔹 Model: base
   🔹 Model: pca
   🔹 Model: hog
▶️  Evaluating blur=5, noise=0...
   🔹 Model: base
   🔹 Model: pca
   🔹 Model: hog
▶️  Evaluating blur=7, noise=0...
   🔹 Model: base
   🔹 Model: pca
   🔹 Model: hog
▶️  Evaluating blur=9, noise=0...
   🔹 Model: base
   🔹 Model: pca
   🔹 Model: hog
▶️  Evaluating blur=19, noise=0...
   🔹 Model: base
   🔹 Model: pca
   🔹 Model: hog
▶️  Evaluating blur=0, noise=0...
   🔹 Model: base
   🔹 Model: pca
   🔹 Model: hog
▶️  Evaluating blur=0, noise=1...
   🔹 Model: base
   🔹 Model: pca
   🔹 Model: hog
▶️  Evaluating blur=0, noise=3...
   🔹 Model: base
  

# Old Stuff

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import json
import shiny_data
import pandas as pd
from PIL import Image, ImageEnhance
from skimage.feature import hog
from sklearn.decomposition import PCA
from IPython.display import display
from sklearn.metrics import classification_report

# suppress warnings for results
import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

# 0 Immune, 1 Other, 2 Stromal, 3 Tumour
Xmat_train, Xmat_val, Xmat_tests_0, Xmat_tests_1, Xmat_tests_2, y_train_enc, y_val_enc, y_tests_enc_0, y_tests_enc_1, y_tests_enc_2 = shiny_data.load_split_images()

def store_results(results, y_true, y_pred, probs=None, blur=0, noise=0, hog=False, pca=False, **extra_metrics):        
    entry = {
        "blur_size": blur,
        "noise_level": noise,
        "HOG": hog,
        "PCA": pca,
        "accuracy": accuracy_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred, average='macro'),
        "precision": precision_score(y_true, y_pred, average='macro'),
        "recall": recall_score(y_true, y_pred, average='macro'),
    }

    f1_classes = f1_score(y_true, y_pred, average=None, zero_division=0)
    precision_classes = precision_score(y_true, y_pred, average=None, zero_division=0)
    recall_classes = recall_score(y_true, y_pred, average=None, zero_division=0)

    label_map = {
        0: "immune",
        1: "other",
        2: "stromal",
        3: "tumour"
    }
    
    for i in range(4):
        label = label_map[i]
        entry[f"f1_{label}"] = f1_classes[i]
        entry[f"precision_{label}"] = precision_classes[i]
        entry[f"recall_{label}"] = recall_classes[i]
    
    if probs is not None:
        confidences = np.max(probs, axis=1)
        entry["confidence_overall"] = np.mean(confidences)
        for cls in range(4):
            label = label_map[cls]
            cls_conf = confidences[y_pred == cls]
            entry[f"confidence_{label}_avg"] = np.mean(cls_conf) if len(cls_conf) > 0 else np.nan
            entry[f"confidence_{label}_std"] = np.std(cls_conf) if len(cls_conf) > 0 else np.nan
    
    total_preds = len(y_pred)
    for cls in range(4):
        label = label_map[cls]
        entry[f"count_pred_{label}"] = np.sum(y_pred == cls)

    # store actual and predicted labels and the confusion matrix as strings
    entry["confusion_matrix"] = str(confusion_matrix(y_true, y_pred, labels=[0, 1, 2, 3]).tolist())

    entry.update(extra_metrics)
    results.append(entry)

def extract_hog_features(images):
    hog_features = []
    for img in images:
        if img.shape[-1] == 3:
            img = Image.fromarray((img * 255).astype(np.uint8)).convert("L")
            img = np.array(img)
        features = hog(img, pixels_per_cell=(8, 8), cells_per_block=(2, 2), feature_vector=True)
        hog_features.append(features)
    return np.array(hog_features)

X_train_flat = Xmat_train.reshape(Xmat_train.shape[0], -1)
X_val_flat   = Xmat_val.reshape(Xmat_val.shape[0], -1)
X_test_flat  = Xmat_test.reshape(Xmat_test.shape[0], -1)

# PCA
pca = PCA(n_components=100)
X_train_pca = pca.fit_transform(X_train_flat)

# hog featues
X_train_hog = extract_hog_features(Xmat_train) 

Tumour images loaded
Immune images loaded
Stromal images loaded
Other images loaded


## Baseline Random Forest

In [ ]:
results = []

# baseline model
model_base = RandomForestClassifier(n_estimators=100, random_state=3888)
model_base.fit(X_train_flat, y_train_enc)
X_test_flat = Xmat_test.reshape(Xmat_test.shape[0], -1)
y_pred = model_base.predict(X_test_flat)
probs = model_base.predict_proba(X_test_flat)
store_results(results, y_test_enc, y_pred, probs=probs)

# baseline PCA model
model_pca = RandomForestClassifier(n_estimators=100, random_state=3888)
model_pca.fit(X_train_pca, y_train_enc)
X_test_pca = pca.transform(X_test_flat)
y_pred = model_pca.predict(X_test_pca)
y_probs = model_pca.predict_proba(X_test_pca)
store_results(results, y_test_enc, y_pred, probs=y_probs)

# HOG baseline model
model_hog = RandomForestClassifier(n_estimators=100, random_state=3888)
model_hog.fit(X_train_hog, y_train_enc)
X_test_hog = extract_hog_features(Xmat_test)
y_pred = model_hog.predict(X_test_hog)
probs = model_hog.predict_proba(X_test_hog)
store_results(results, y_test_enc, y_pred, probs=probs, hog=True)

# blur
for radius in [0, 1, 3, 5, 7, 9, 19]:
    X_blur = shiny_data.apply_blur(Xmat_test, radius)
    X_test_flat = X_blur.reshape(X_blur.shape[0], -1)
    y_pred = model_base.predict(X_test_flat)
    probs = model_base.predict_proba(X_test_flat)
    store_results(results, y_test_enc, y_pred, probs=probs, blur=radius)

    X_blur = shiny_data.apply_blur(Xmat_test, radius)
    X_test_flat = X_blur.reshape(X_blur.shape[0], -1)
    X_test_pca = pca.transform(X_test_flat)
    y_pred = model_pca.predict(X_test_pca)
    y_probs = model_pca.predict_proba(X_test_pca)
    store_results(results, y_test_enc, y_pred, blur=radius, probs=y_probs)

    X_blur = shiny_data.apply_blur(Xmat_test, radius)
    X_test_hog = extract_hog_features(X_blur)
    y_pred = model_hog.predict(X_test_hog)
    probs = model_hog.predict_proba(X_test_hog)
    store_results(results, y_test_enc, y_pred, probs=probs, blur=radius, hog=True)

# noise
for noise_std in [0, 1, 3, 5, 10, 20, 30]:
    np.random.seed(3888 + noise_std)
    X_noisy = shiny_data.apply_noise(Xmat_test, std=noise_std)
    X_test_flat = X_noisy.reshape(X_noisy.shape[0], -1)
    y_pred = model_base.predict(X_test_flat)
    probs = model_base.predict_proba(X_test_flat)
    store_results(results, y_test_enc, y_pred, probs=probs, noise=noise_std)

    np.random.seed(3888 + noise_std)  # set seed globally
    X_noisy = shiny_data.apply_noise(Xmat_test, std=noise_std)  # no seed param
    X_test_flat = X_noisy.reshape(X_noisy.shape[0], -1)
    X_test_pca = pca.transform(X_test_flat)
    y_pred = model_pca.predict(X_test_pca)
    y_probs = model_pca.predict_proba(X_test_pca)
    store_results(results, y_test_enc, y_pred, noise=noise_std, probs=y_probs)

    np.random.seed(3888 + noise_std)
    X_noisy = shiny_data.apply_noise(Xmat_test, std=noise_std)
    X_test_hog = extract_hog_features(X_noisy)
    y_pred = model_hog.predict(X_test_hog)
    probs = model_hog.predict_proba(X_test_hog)
    store_results(results, y_test_enc, y_pred, probs=probs, noise=noise_std, hog=True)

# blur and noise
for radius in [0, 1, 3, 5, 7, 9, 19]:
    for noise_std in [0, 1, 3, 5, 10, 20, 30]:
        np.random.seed(10000 + radius * 100 + noise_std)
        X_blur = shiny_data.apply_blur(Xmat_test, radius)
        X_combo = shiny_data.apply_noise(X_blur, std=noise_std)
        X_test_flat = X_combo.reshape(X_combo.shape[0], -1)
        y_pred = model_base.predict(X_test_flat)
        probs = model_base.predict_proba(X_test_flat)
        store_results(results, y_test_enc, y_pred, probs=probs, blur=radius, noise=noise_std)

        np.random.seed(10000 + radius * 100 + noise_std)  # set seed globally
        X_blur = shiny_data.apply_blur(Xmat_test, radius)
        X_combo = shiny_data.apply_noise(X_blur, std=noise_std)
        X_test_flat = X_combo.reshape(X_combo.shape[0], -1)
        X_test_pca = pca.transform(X_test_flat)
        y_pred = model_pca.predict(X_test_pca)
        y_probs = model_pca.predict_proba(X_test_pca)
        store_results(results, y_test_enc, y_pred, blur=radius, noise=noise_std, probs=y_probs)

        np.random.seed(10000 + radius * 100 + noise_std)
        X_blur = shiny_data.apply_blur(Xmat_test, radius)
        X_combo = shiny_data.apply_noise(X_blur, std=noise_std)
        X_test_hog = extract_hog_features(X_combo)
        y_pred = model_hog.predict(X_test_hog)
        probs = model_hog.predict_proba(X_test_hog)
        store_results(results, y_test_enc, y_pred, probs=probs, blur=radius, noise=noise_std, hog=True)

## PCA Random Forest

In [3]:
model_pca = RandomForestClassifier(n_estimators=100, random_state=3888)
model_pca.fit(X_train_pca, y_train_enc)

# baseline
X_test_pca = pca.transform(X_test_flat)
y_pred = model_pca.predict(X_test_pca)
y_probs = model_pca.predict_proba(X_test_pca)
store_results(results, y_test_enc, y_pred, probs=y_probs)

for radius in [1, 3, 5, 7, 9, 19]: # blur
    X_blur = shiny_data.apply_blur(Xmat_test, radius)
    X_test_flat = X_blur.reshape(X_blur.shape[0], -1)
    X_test_pca = pca.transform(X_test_flat)
    y_pred = model_pca.predict(X_test_pca)
    y_probs = model_pca.predict_proba(X_test_pca)
    store_results(results, y_test_enc, y_pred, blur=radius, probs=y_probs)

for noise_std in [1, 3, 5, 10, 20, 30]:
    np.random.seed(3888 + noise_std)  # set seed globally
    X_noisy = shiny_data.apply_noise(Xmat_test, std=noise_std)  # no seed param
    X_test_flat = X_noisy.reshape(X_noisy.shape[0], -1)
    X_test_pca = pca.transform(X_test_flat)
    y_pred = model_pca.predict(X_test_pca)
    y_probs = model_pca.predict_proba(X_test_pca)
    store_results(results, y_test_enc, y_pred, noise=noise_std, probs=y_probs)

for radius in [1, 3, 5, 7, 9, 19]:
    for noise_std in [1, 3, 5, 10, 20, 30]:
        np.random.seed(10000 + radius * 100 + noise_std)  # set seed globally
        X_blur = shiny_data.apply_blur(Xmat_test, radius)
        X_combo = shiny_data.apply_noise(X_blur, std=noise_std)
        X_test_flat = X_combo.reshape(X_combo.shape[0], -1)
        X_test_pca = pca.transform(X_test_flat)
        y_pred = model_pca.predict(X_test_pca)
        y_probs = model_pca.predict_proba(X_test_pca)
        store_results(results, y_test_enc, y_pred, blur=radius, noise=noise_std, probs=y_probs)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-pa


=== After Augmentations ===


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,hog,pca,Blur,Contrast,accuracy,f1,precision,recall,f1_class_0,f1_class_1,f1_class_2,f1_class_3,avg_conf,avg_conf_class_0,avg_conf_class_1,avg_conf_class_2,avg_conf_class_3
0,False,False,0,0.00,0.52975,0.519056,0.522985,0.52975,0.602448,0.326599,0.505759,0.641416,0.421730,0.365404,0.356304,0.480110,0.455003
1,False,False,0,1.25,0.25000,0.100000,0.062500,0.25000,0.000000,0.000000,0.000000,0.400000,0.420000,NaN,NaN,NaN,0.420000
2,False,False,0,1.50,0.25000,0.100000,0.062500,0.25000,0.000000,0.000000,0.000000,0.400000,0.420000,NaN,NaN,NaN,0.420000
3,False,False,0,1.75,0.25000,0.100000,0.062500,0.25000,0.000000,0.000000,0.000000,0.400000,0.420000,NaN,NaN,NaN,0.420000
4,False,False,0,2.00,0.25000,0.100000,0.062500,0.25000,0.000000,0.000000,0.000000,0.400000,0.420000,NaN,NaN,NaN,0.420000
5,False,False,1,0.00,0.53100,0.519989,0.531346,0.53100,0.590017,0.341629,0.512233,0.636076,0.432130,0.365939,0.357240,0.485168,0.471721
6,False,False,1,1.25,0.25000,0.100000,0.062500,0.25000,0.000000,0.000000,0.000000,0.400000,0.420000,NaN,NaN,NaN,0.420000
7,False,False,1,1.50,0.25000,0.100000,0.062500,0.25000,0.000000,0.000000,0.000000,0.400000,0.420000,NaN,NaN,NaN,0.420000
8,False,False,1,1.75,0.25000,0.100000,0.062500,0.25000,0.000000,0.000000,0.000000,0.400000,0.420000,NaN,NaN,NaN,0.420000
9,False,False,1,2.00,0.25000,0.100000,0.062500,0.25000,0.000000,0.000000,0.000000,0.400000,0.420000,NaN,NaN,NaN,0.420000


## HOG Random Forest

In [7]:
model_hog = RandomForestClassifier(n_estimators=100, random_state=3888)
model_hog.fit(X_train_hog, y_train_enc)

# baseline
X_test_hog = extract_hog_features(Xmat_test)
y_pred = model_hog.predict(X_test_hog)
probs = model_hog.predict_proba(X_test_hog)
store_results(results, y_test_enc, y_pred, probs=probs, hog=True)

# blur
for radius in [1, 3, 5, 7, 9, 19]:
    X_blur = shiny_data.apply_blur(Xmat_test, radius)
    X_test_hog = extract_hog_features(X_blur)
    y_pred = model_hog.predict(X_test_hog)
    probs = model_hog.predict_proba(X_test_hog)
    store_results(results, y_test_enc, y_pred, probs=probs, blur=radius, hog=True)

# noise
for noise_std in [1, 3, 5, 10, 20, 30]:
    np.random.seed(3888 + noise_std)
    X_noisy = shiny_data.apply_noise(Xmat_test, std=noise_std)
    X_test_hog = extract_hog_features(X_noisy)
    y_pred = model_hog.predict(X_test_hog)
    probs = model_hog.predict_proba(X_test_hog)
    store_results(results, y_test_enc, y_pred, probs=probs, noise=noise_std, hog=True)

# blur and noise
for radius in [1, 3, 5, 7, 9, 19]:
    for noise_std in [1, 3, 5, 10, 20, 30]:
        np.random.seed(10000 + radius * 100 + noise_std)
        X_blur = shiny_data.apply_blur(Xmat_test, radius)
        X_combo = shiny_data.apply_noise(X_blur, std=noise_std)
        X_test_hog = extract_hog_features(X_combo)
        y_pred = model_hog.predict(X_test_hog)
        probs = model_hog.predict_proba(X_test_hog)
        store_results(results, y_test_enc, y_pred, probs=probs, blur=radius, noise=noise_std, hog=True)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-pa


=== After Augmentations ===


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,hog,pca,Blur,Contrast,accuracy,f1,precision,recall,f1_class_0,f1_class_1,f1_class_2,f1_class_3,avg_conf,avg_conf_class_0,avg_conf_class_1,avg_conf_class_2,avg_conf_class_3
0,False,False,0,0.00,0.52975,0.519056,0.522985,0.52975,0.602448,0.326599,0.505759,0.641416,0.421730,0.365404,0.356304,0.480110,0.455003
1,False,False,0,1.25,0.25000,0.100000,0.062500,0.25000,0.000000,0.000000,0.000000,0.400000,0.420000,NaN,NaN,NaN,0.420000
2,False,False,0,1.50,0.25000,0.100000,0.062500,0.25000,0.000000,0.000000,0.000000,0.400000,0.420000,NaN,NaN,NaN,0.420000
3,False,False,0,1.75,0.25000,0.100000,0.062500,0.25000,0.000000,0.000000,0.000000,0.400000,0.420000,NaN,NaN,NaN,0.420000
4,False,False,0,2.00,0.25000,0.100000,0.062500,0.25000,0.000000,0.000000,0.000000,0.400000,0.420000,NaN,NaN,NaN,0.420000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,True,False,10,0.00,0.27825,0.196837,0.227026,0.27825,0.402404,0.143590,0.241356,0.000000,0.357600,0.363209,0.332603,0.339495,NaN
86,True,False,10,1.25,0.26875,0.176411,0.224271,0.26875,0.399451,0.131250,0.174944,0.000000,0.361403,0.366571,0.330893,0.335960,NaN
87,True,False,10,1.50,0.26400,0.163683,0.216926,0.26400,0.400000,0.103668,0.151062,0.000000,0.364415,0.369563,0.327205,0.333284,NaN
88,True,False,10,1.75,0.26350,0.162246,0.469811,0.26350,0.397871,0.090312,0.158805,0.001998,0.365582,0.370881,0.323486,0.331250,0.290000


In [12]:
df.to_csv("model_results.csv", index=False)